
# **Gen** ret<strong><font color="red">AI</font></strong>l

Customer search:
* I search flower for **weading.**
* I search flower for **mother day.**
* I search flower for **valentine day.**

Company provide:
* I found the same product <strong><font color="red">contextualized</font></strong> to my search

In [1]:
ZERO_SHOT_PROMPT = '''Product description **PRODUCT** according the **THEMA**
'''

ZERO_SHOT_PROMPT = '''Why this **PRODUCT** is perfect for the special event: **THEMA**
'''

# Init project

In [2]:
# init the google cloud project id

def init_gcloud_(project_id = "ucs-fishfood-8"):
    """
    Initialize the google cloud project id.
    """
    # [START gae_python37_app_init_gcloud]
    !gcloud config set project $project_id

def init_project_genappbuilder():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "ucs-fishfood-8")


def init_project_images():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "cloud-llm-preview4")


def init_project_workstations():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "cloud-workstations-demo-2")

def init_project_ml_api():
    """
    Initialize the google cloud project id.
    """
    init_gcloud_(project_id = "google.com:ml-baguette-demos")

    

# Image Gen (not used in the demo)

In [7]:
PROJECT_ID = "google.com:ml-baguette-demos"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "text-bison@001" # @param {type:"string"}

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Initialize Vertex AI SDK
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

!curl -X POST \
-H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/ui/projects/google.com:ml-baguette-demos/locations/us-central1/models:upload \
-d '{"model": {"display_name": "vision-generative-model","large_model_reference": {"name": "imagegeneration-001"}}}'

Vertex AI SDK version: 1.25.0
{
  "name": "projects/660199673046/locations/us-central1/operations/5344077516917178368",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.aiplatform.ui.UploadModelOperationMetadata",
    "genericMetadata": {
      "createTime": "2023-05-31T11:24:38.821446Z",
      "updateTime": "2023-05-31T11:24:38.821446Z",
      "state": "RUNNING",
      "worksOn": [
        "projects/660199673046/locations/us-central1/models/8074085517689356288@1"
      ]
    }
  }
}


In [8]:
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()

ACCESS_TOKEN = gcloud_token[0]

# This endpoint is what was available from the Private API Documentation
# https://cloud.devsite.corp.google.com/vision-ai/docs/priv/generate-images-api
ENDPOINT_URL = 'projects/66194518407/locations/us-central1/endpoints/4549422873969688576'

# This endpoint below is from the Internal Fishfooding Guide: go/IG-fishfooding
FF_ENDPOINT_URL = 'projects/cloud-lvm-fishfooding/locations/us-central1/endpoints/6988653136307027968'

In [9]:
import requests
import json
import time

def image_gen_model(prompt, sampleImageSize, sampleCount, endpointType='Prod', seed=None):
  headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
  }
  # Advanced option, try different the seed numbers
  # any random integer number range: (0, 2147483647)
  if seed==None:
    data = {"instances": [{"prompt": prompt}],"parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount}}
  else:
    # Use & provide a seed, if possible, so that we can reproduce the results when needed.
    data = {"instances": [{"prompt": prompt}],"parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount, "seed": seed}}

  print(data)
  if endpointType=='Prod':
    # Prod usage
    response = requests.post(f'https://us-central1-aiplatform.googleapis.com/v1/{ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
  else:
    # Autopush usage
    response = requests.post(f'https://us-central1-autopush-aiplatform.sandbox.googleapis.com/v1/{FF_ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
  return response

In [10]:
#@title Click `Show code` in the code cell (Don't forget to run this cell). { display-mode: "form" }

sampleImageSize = '1024' #@param {type:"string"}
sampleCount = 6 #@param {type:"integer"}
prompt = 'audacious and whimsical fantasy house shaped like pasion fruit with windows and door, in the desert' #@param {type:"string"}
endpointType = "Fishfood" #@param ["Prod", "Fishfood"]

# Advanced option, try different the seed numbers
# any random integer number range: (0, 2147483647)
seed = None #@param {type:"raw"}

print('sampleImageSize:', sampleImageSize)
print('sampleCount:', sampleCount)
print('prompt', prompt)
print('endpointType', endpointType)
print('seed', seed)

sampleImageSize: 1024
sampleCount: 6
prompt audacious and whimsical fantasy house shaped like pasion fruit with windows and door, in the desert
endpointType Fishfood
seed None


In [11]:
# Use & provide a seed, if possible, so that we can reproduce the results when needed.
response = image_gen_model(prompt, sampleImageSize, sampleCount, endpointType, seed)
json_response = json.loads(response.text)

try:
  predictions = json_response['predictions']
except:
  print("An error occured calling the API.")
  print("1. Check if response was not blocked based on policy violation, check if the UI behaves the same way...")
  print("2. Try a different prompt to see if that was the problem.\n")
  print(response.text)
  # print(dir(response))

{'instances': [{'prompt': 'audacious and whimsical fantasy house shaped like pasion fruit with windows and door, in the desert'}], 'parameters': {'sampleImageSize': '1024', 'sampleCount': 6}}
An error occured calling the API.
1. Check if response was not blocked based on policy violation, check if the UI behaves the same way...
2. Try a different prompt to see if that was the problem.

{
  "error": {
    "code": 403,
    "message": "Permission 'aiplatform.endpoints.predict' denied on resource '//aiplatform.googleapis.com/projects/cloud-lvm-fishfooding/locations/us-central1/endpoints/6988653136307027968' (or it may not exist).",
    "status": "PERMISSION_DENIED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "IAM_PERMISSION_DENIED",
        "domain": "aiplatform.googleapis.com",
        "metadata": {
          "permission": "aiplatform.endpoints.predict",
          "resource": "projects/cloud-lvm-fishfooding/locations/us-central1

In [178]:
import requests
import json
import time

def image_gen_model(prompt, sampleImageSize, sampleCount, endpointType='Prod', seed=None):
  init_project_images()
  gcloud_token = !gcloud auth print-access-token
  gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
  ACCESS_TOKEN = gcloud_token[0]

  # This endpoint is what was available from the Private API Documentation
  # https://cloud.devsite.corp.google.com/vision-ai/docs/priv/generate-images-api
  ENDPOINT_URL = 'projects/66194518407/locations/us-central1/endpoints/4549422873969688576'

  # This endpoint below is from the Internal Fishfooding Guide: go/IG-fishfooding
  #FF_ENDPOINT_URL = 'projects/cloud-lvm-fishfooding/locations/us-central1/endpoints/6988653136307027968'

  headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
  }
  # Advanced option, try different the seed numbers
  # any random integer number range: (0, 2147483647)
  if seed==None:
    data = {"instances": [{"prompt": prompt}],"parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount}}
  else:
    # Use & provide a seed, if possible, so that we can reproduce the results when needed.
    data = {"instances": [{"prompt": prompt}],"parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount, "seed": seed}}

  print(data)
  #if endpointType=='Prod':
    # Prod usage
  response = requests.post(f'https://us-central1-aiplatform.googleapis.com/v1/{ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
  #else:
    # Autopush usage
  #  response = requests.post(f'https://us-central1-autopush-aiplatform.sandbox.googleapis.com/v1/{FF_ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
  return response

In [179]:
#@title Click `Show code` in the code cell (Don't forget to run this cell). { display-mode: "form" }

sampleImageSize = '1024' #@param {type:"string"}
sampleCount = 6 #@param {type:"integer"}
prompt = 'audacious and whimsical fantasy house shaped like pasion fruit with windows and door, in the desert' #@param {type:"string"}
endpointType = "Prod" #@param ["Prod", "Fishfood"]

# Advanced option, try different the seed numbers
# any random integer number range: (0, 2147483647)
seed = None #@param {type:"raw"}

print('sampleImageSize:', sampleImageSize)
print('sampleCount:', sampleCount)
print('prompt', prompt)
print('endpointType', endpointType)
print('seed', seed)

# Use & provide a seed, if possible, so that we can reproduce the results when needed.
response = image_gen_model(prompt, sampleImageSize, sampleCount, endpointType, seed)
json_response = json.loads(response.text)

try:
  predictions = json_response['predictions']
except:
  print("An error occured calling the API.")
  print("1. Check if response was not blocked based on policy violation, check if the UI behaves the same way...")
  print("2. Try a different prompt to see if that was the problem.\n")
  print(response.text)
  # print(dir(response))

sampleImageSize: 1024
sampleCount: 6
prompt audacious and whimsical fantasy house shaped like pasion fruit with windows and door, in the desert
endpointType Prod
seed None
Updated property [core/project].
{'instances': [{'prompt': 'audacious and whimsical fantasy house shaped like pasion fruit with windows and door, in the desert'}], 'parameters': {'sampleImageSize': '1024', 'sampleCount': 6}}
An error occured calling the API.
1. Check if response was not blocked based on policy violation, check if the UI behaves the same way...
2. Try a different prompt to see if that was the problem.

{
  "error": {
    "code": 403,
    "message": "Permission 'aiplatform.endpoints.predict' denied on resource '//aiplatform.googleapis.com/projects/66194518407/locations/us-central1/endpoints/4549422873969688576' (or it may not exist).",
    "status": "PERMISSION_DENIED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "IAM_PERMISSION_DENIED",
     

# Palm LLM 

#### The `top_k` parameter (range: 0.0 - 40, default 40)

##### What is _top_k_?
`top_k` changes how the model selects tokens for output. A `top_k` of 1 means the selected token is the most probable among all tokens in the model's vocabulary (also called greedy decoding). In contrast, a `top_k` of 3 means that the next token is selected from the top 3 most probable tokens (using temperature). For each token selection step, the `top_k` tokens with the highest probabilities are sampled. Then tokens are further filtered based on `top_p` with the final token selected using temperature sampling.

##### How does _top_k_ affect the response?

Specify a lower value for less random responses and a higher value for more random responses.

For more information on the `top_k` parameter for text models, please refer to the [documentation on model parameters](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#text_model_parameters).
                                                                                                                       
#### The `top_p` parameter (range: 0.0 - 1.0, default 0.95)
                                                                                                                       

In [3]:
from vertexai.preview.language_models import TextGenerationModel,\
                                            ChatModel,\
                                            InputOutputTextPair,\
                                            TextEmbeddingModel

from IPython.display import display, Markdown
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

generation_model = TextGenerationModel.from_pretrained("text-bison@001")



def generative_example(text_input, temperature=0.6, top_k=40, top_p=0.9, max_output_tokens=512):
    """Text Generation with a Large Language Model."""
    response = generation_model.predict(
        text_input,
        temperature=temperature,
        top_k=top_k, # default top_k=1
        top_p=top_p, # default top_p=0
        max_output_tokens=max_output_tokens, # default 32
    )
    #print(response.text)
    
    return response

generative_example(ZERO_SHOT_PROMPT.replace('**PRODUCT**',"flower").replace('**THEMA**',"weading") )


Roses are a classic flower for weddings, and for good reason. They are beautiful, elegant, and symbolize love, romance, and commitment. Roses come in a variety of colors, so you can choose the perfect shade to match your wedding theme. You can also choose roses in different sizes, shapes, and arrangements to create a unique and personalized bouquet.

# Cloud vision to enrich text prompt

In [4]:
from typing import Sequence

from google.cloud import vision


def analyze_image_from_uri(
    image_uri: str,
    feature_types: Sequence,
) -> vision.AnnotateImageResponse:
    client = vision.ImageAnnotatorClient()

    image = vision.Image()
    image.source.image_uri = image_uri
    features = [vision.Feature(type_=feature_type) for feature_type in feature_types]
    request = vision.AnnotateImageRequest(image=image, features=features)

    response = client.annotate_image(request=request)

    return response


def print_labels(response: vision.AnnotateImageResponse):
    print("LABELS " +"==" * 80)
    for label in response.label_annotations:
        print(
            f"{label.score:4.0%}",
            f"{label.description:5}",
            sep=" | ",
        )
    return response.label_annotations

def print_text(response: vision.AnnotateImageResponse):
    print("TEXT "+"==" * 80)
    for annotation in response.text_annotations:
        vertices = [f"({v.x},{v.y})" for v in annotation.bounding_poly.vertices]
        print(
            f"{repr(annotation.description):42}",
            ",".join(vertices),
            sep=" | ",
        )
        
def print_landmarks(response: vision.AnnotateImageResponse, min_score: float = 0.5):
    print("LANDMARKS " +"==" * 80)
    for landmark in response.landmark_annotations:
        if landmark.score < min_score:
            continue
        vertices = [f"({v.x},{v.y})" for v in landmark.bounding_poly.vertices]
        lat_lng = landmark.locations[0].lat_lng
        print(
            f"{landmark.description:18}",
            ",".join(vertices),
            f"{lat_lng.latitude:.5f}",
            f"{lat_lng.longitude:.5f}",
            sep=" | ",
        )       

def print_objects(response: vision.AnnotateImageResponse):
    print("OBJECTS " +"==" * 80)
    for obj in response.localized_object_annotations:
        nvertices = obj.bounding_poly.normalized_vertices
        print(
            f"{obj.score:4.0%}",
            f"{obj.name:15}",
            f"{obj.mid:10}",
            ",".join(f"({v.x:.1f},{v.y:.1f})" for v in nvertices),
            sep=" | ",
        )   

def print_logos(response: vision.AnnotateImageResponse, min_score: float = 0.5):
    print("LOGO "+"==" * 80)

    for logo in response.logo_annotations:
        if logo.score < min_score:
            continue
        
        print('Logos:')
        print(logo.description)
    
def print_safe(response: vision.AnnotateImageResponse):
    print("SAFE "+"==" * 80)

    safe = response.safe_search_annotation

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Safe search:')

    print(f'adult: {likelihood_name[safe.adult]}')
    print(f'medical: {likelihood_name[safe.medical]}')
    print(f'spoofed: {likelihood_name[safe.spoof]}')
    print(f'violence: {likelihood_name[safe.violence]}')
    print(f'racy: {likelihood_name[safe.racy]}')

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))    
    

def print_web(response: vision.AnnotateImageResponse):
    print("WEB "+"==" * 80)

    annotations = response.web_detection

    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print(f'\nBest guess label: {label.label}')

    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images found:'.format(
            len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            print(f'\n\tPage url   : {page.url}')

            if page.full_matching_images:
                print('\t{} Full Matches found: '.format(
                       len(page.full_matching_images)))

                for image in page.full_matching_images:
                    print(f'\t\tImage url  : {image.url}')

            if page.partial_matching_images:
                print('\t{} Partial Matches found: '.format(
                       len(page.partial_matching_images)))

                for image in page.partial_matching_images:
                    print(f'\t\tImage url  : {image.url}')

    if annotations.web_entities:
        print('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print(f'\n\tScore      : {entity.score}')
            print(f'\tDescription: {entity.description}')

    if annotations.visually_similar_images:
        print('\n{} visually similar images found:\n'.format(
            len(annotations.visually_similar_images)))

        for image in annotations.visually_similar_images:
            print(f'\tImage url    : {image.url}')

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [5]:
def image_labels (
image_uri = "https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/2FRI-GALLERY-0-1.jpg?frz-width=643&context=bWFzdGVyfGltYWdlc3wyMDkwMjJ8aW1hZ2UvanBlZ3xpbWFnZXMvaGJjL2g4Ny85NTMwNjA5NTk4NDk0LmpwZ3xlM2Y0NmE0YWY3YjQ3MjcyMzRhNmEwOTE3MjY5OTMxOWU1OGQ4ZmU0MWM2ZTI3NDRkODAzNTllOWYyMzVkMTRm"
):
    features = [vision.Feature.Type.LABEL_DETECTION, 
                vision.Feature.Type.LANDMARK_DETECTION, 
                vision.Feature.Type.LOGO_DETECTION,
    #            vision.Feature.Type.SAFE_SEARCH_DETECTION,
                vision.Feature.Type.WEB_DETECTION,
                vision.Feature.Type.TEXT_DETECTION,
                vision.Feature.Type.OBJECT_LOCALIZATION]

    features = [vision.Feature.Type.LABEL_DETECTION] 

    response = analyze_image_from_uri(image_uri, features)
    labels = print_labels(response)
    list_labels = [x.description for x in list(filter(lambda x: (x.score > 0.80 ), labels)) ]

    return list_labels

image_labels()

LABELS ================================================================================================================================================================
 98% | Flower
 95% | Plant
 92% | Vase 
 92% | Flowerpot
 91% | Petal
 83% | Hybrid tea rose
 83% | Creative arts
 83% | Pink 
 80% | Rose 
 79% | Flower Arranging


['Flower',
 'Plant',
 'Vase',
 'Flowerpot',
 'Petal',
 'Hybrid tea rose',
 'Creative arts',
 'Pink']

# Translate API to support multiple input/output languages (only english is supported by Palm actually)
    

In [18]:
!pip install google-cloud-translate==2.0.1

In [6]:
def translate_text(target, text):

    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print("Text: {}".format(result["input"]))
    print("Translation: {}".format(result["translatedText"]))
    print("Detected source language: {}".format(result["detectedSourceLanguage"]))
    
    if isinstance(result["translatedText"], bytes):
        print("bytes")
        text = result["translatedText"].decode("utf-8")
        return text
        
    unicode_string = result["translatedText"]
    
    return clean_text(unicode_string)
    
# TODO: Clean me ugly code
def clean_text(s : str):    
    return s.replace("&#39;", "'")

init_project_ml_api()
translate_text( "FR", "The tree is perfect for the special event: wedding because it is covered in beautiful pink flowers. The flowers are a symbol of love and romance, and they make the tree look very elegant and festive. The tree would be a beautiful addition to any wedding ceremony or reception, and it would make a lasting impression on guests.")



Updated property [core/project].
Text: The tree is perfect for the special event: wedding because it is covered in beautiful pink flowers. The flowers are a symbol of love and romance, and they make the tree look very elegant and festive. The tree would be a beautiful addition to any wedding ceremony or reception, and it would make a lasting impression on guests.
Translation: L&#39;arbre est parfait pour l&#39;événement spécial : mariage car il est couvert de belles fleurs roses. Les fleurs sont un symbole d&#39;amour et de romance, et elles donnent à l&#39;arbre un aspect très élégant et festif. L&#39;arbre serait un bel ajout à toute cérémonie de mariage ou réception, et il ferait une impression durable sur les invités.
Detected source language: en


"L'arbre est parfait pour l'événement spécial : mariage car il est couvert de belles fleurs roses. Les fleurs sont un symbole d'amour et de romance, et elles donnent à l'arbre un aspect très élégant et festif. L'arbre serait un bel ajout à toute cérémonie de mariage ou réception, et il ferait une impression durable sur les invités."

# gradio interface used by the demo

In [20]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install gradio==3.24

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 33.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gradio
    Found existing installation: gradio 3.32.0
    Uninstalling gradio-3.32.0:
      Successfully uninstalled gradio-3.32.0
Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install --user --upgrade gradio

  Using cached gradio-3.32.0-py3-none-any.whl (19.9 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [25]:
from gradio_client import Client

genappbuilder_demo_url = "https://c5c9fecc7e8a479b1e.gradio.live"
client_genappbuilder = Client(genappbuilder_demo_url)

client_genappbuilder.view_api()

client_genappbuilder.predict("fleur",  fn_index=0)


Loaded as API: https://c5c9fecc7e8a479b1e.gradio.live ✔
Client.predict() Usage Info
---------------------------
Named API endpoints: 0

Unnamed API endpoints: 1

 - predict(query, fn_index=0) -> (title, link, snippet, description, image)
    Parameters:
     - [Textbox] query: str 
    Returns:
     - [Textbox] title: str 
     - [Textbox] link: str 
     - [Textbox] snippet: str 
     - [Textbox] description: str 
     - [Textbox] image: str 



('Fleurs Deuil - Livraison fleurs décès, deuil, enterrement',
 'https://www.interflora.fr/c/fleurs-deuil',
 'Livraison de fleurs pour deuil, enterrement | Livraison sur tombe ou sur lieu de cérémonie en - de 4h | Qualité garantie : Bouquet réalisé par un artisan\xa0...',
 'Livraison de fleurs pour deuil, enterrement | Livraison sur tombe ou sur lieu de cérémonie en - de 4h | Qualité garantie : Bouquet réalisé par un artisan fleuriste',
 'https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/6INT-THUMBMAY-PRIMARY-0-1.png?context=bWFzdGVyfGltYWdlc3wzNDk1MDN8aW1hZ2UvcG5nfGltYWdlcy9oZTYvaDU5Lzk0Njg0MjQ4ODAxNTgucG5nfGFkOGQwMTc5NjQzNmZjNDM0MmFhY2Q4MjBmZWI3YzI3N2UwNTg4OGNjNGI4OTZhM2JjZmI5MDNmNTBmYmFjMDM')

In [10]:
html = """
<div class="PushMomentsFirst-picture"><picture class="BlockPicture BlockResizePicture"><source srcset="https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/Bandeau-principal-MothersDay2023-Octopus-02.jpg?frz-width=640&amp;frz-height=450&amp;frz-gravity=center&amp;frz-enlarge=true&amp;context=bWFzdGVyfHJvb3R8MjIwMjIwfGltYWdlL2pwZWd8aGIwL2gzZC85NTc5NzE5OTgzMTM0LmpwZ3wxMDg1YjM1Y2E0NDkzMDk1NWRhMTg0ODViNWU5YzBlYTk2OTk2ZDI3OTE5ODc2MjNiY2M3NDMyYmZjNTdjMGJl 1x" media="(max-width: 639px)"><source srcset="https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/Bandeau-principal-MothersDay2023-Octopus-02.jpg?frz-width=959&amp;frz-height=450&amp;frz-gravity=center&amp;frz-enlarge=true&amp;context=bWFzdGVyfHJvb3R8MjIwMjIwfGltYWdlL2pwZWd8aGIwL2gzZC85NTc5NzE5OTgzMTM0LmpwZ3wxMDg1YjM1Y2E0NDkzMDk1NWRhMTg0ODViNWU5YzBlYTk2OTk2ZDI3OTE5ODc2MjNiY2M3NDMyYmZjNTdjMGJl 1x" media="(max-width: 959px)"><source srcset="https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/Bandeau-principal-MothersDay2023-Octopus-02.jpg?frz-width=1600&amp;frz-height=450&amp;frz-gravity=center&amp;frz-enlarge=true&amp;context=bWFzdGVyfHJvb3R8MjIwMjIwfGltYWdlL2pwZWd8aGIwL2gzZC85NTc5NzE5OTgzMTM0LmpwZ3wxMDg1YjM1Y2E0NDkzMDk1NWRhMTg0ODViNWU5YzBlYTk2OTk2ZDI3OTE5ODc2MjNiY2M3NDMyYmZjNTdjMGJl 1x" media="(min-width: 960px)"> <img src="https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/Bandeau-principal-MothersDay2023-Octopus-02.jpg?context=bWFzdGVyfHJvb3R8MjIwMjIwfGltYWdlL2pwZWd8aGIwL2gzZC85NTc5NzE5OTgzMTM0LmpwZ3wxMDg1YjM1Y2E0NDkzMDk1NWRhMTg0ODViNWU5YzBlYTk2OTk2ZDI3OTE5ODc2MjNiY2M3NDMyYmZjNTdjMGJl" alt="Fête des Mères Dimanche 4 juin" width="480" height="480" fetchpriority="high"></picture></div>
"""

In [33]:
import gradio as gr
import base64
import numpy as np
import math
from matplotlib import pyplot as plt
import requests

gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()

ACCESS_TOKEN = gcloud_token[0]

endpointType = "Prod" #@param ["Prod", "Fishfood"]
seed=0

def update(search,
           theme,
           language,
           temperature,
           top_k,
           top_p,
           max_output_tokens,
           endpointType='Prod' #,           seed=None
          ):
    
    title, link, snippet, description, image = client_genappbuilder.predict(search,  fn_index=0)
    print("Title: ", title)
    print("Link: ", link)
    print("Description: ", description)
    print("Snippet: ", snippet)
    print("Image: ", image)
    
    images = []
  
    images.append(Image.open(requests.get(image , stream=True).raw))

    #image_uri = "https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/2FRI-GALLERY-0-1.jpg?frz-width=643&context=bWFzdGVyfGltYWdlc3wyMDkwMjJ8aW1hZ2UvanBlZ3xpbWFnZXMvaGJjL2g4Ny85NTMwNjA5NTk4NDk0LmpwZ3xlM2Y0NmE0YWY3YjQ3MjcyMzRhNmEwOTE3MjY5OTMxOWU1OGQ4ZmU0MWM2ZTI3NDRkODAzNTllOWYyMzVkMTRm"

    list_labels = image_labels(image)
    prompt = (ZERO_SHOT_PROMPT.replace('**PRODUCT**',search).replace('**THEMA**',theme) + " with the labels: "+ ", ".join(list_labels))
    
    
    print(prompt)
    print(temperature)
    print(float(temperature))
    response = generative_example(prompt, temperature=float(temperature), top_k=top_k, top_p=top_p, max_output_tokens=max_output_tokens)

    print(response.text)
    
    if response._prediction_response[0][0]['safetyAttributes']['blocked'] == True:
        print("Function not available")
    else:
        print("LLM answer: \n",response.text)    

    import pandas as pd
    safety_categories = response._prediction_response[0][0]['safetyAttributes']['categories']
    safety_scores = response._prediction_response[0][0]['safetyAttributes']['scores']
    safety_blocked = response._prediction_response[0][0]['safetyAttributes']['blocked']

    safety_attributes_df = pd.DataFrame(list(zip(safety_categories, safety_scores)), columns =['categories', 'scores'])
    print(safety_attributes_df)

    
    if len(language) > 1:
        desc = translate_text(language, response.text)
    else:
        desc = response.text

    return desc, "composition:" + ", ".join(list_labels) , images

## TODO: Ajouter bannière
# bugfix utf8
# affichage image fleur
import gradio as gr



theme_demo = gr.themes.Soft(
    secondary_hue="blue",
    font=[gr.themes.GoogleFont('Source Sans Pro'), 'ui-sans-serif', 'system-ui', 'sans-serif'],
).set(
    background_fill_primary='white',
    shadow_drop='rgba(0,0,0,0.05) 0px 1px 2px 0px',
    shadow_drop_lg='0 1px 3px 0 rgb(0 0 0 / 0.1), 0 1px 2px -1px rgb(0 0 0 / 0.1)',
    shadow_spread='3px',
    block_background_fill='*background_fill_primary',
    block_border_width='1px',
    block_border_width_dark='1px',
    block_label_background_fill='*background_fill_primary',
    block_label_background_fill_dark='*background_fill_secondary',
    block_label_text_color='*neutral_500',
    block_label_text_color_dark='*neutral_200',
    block_label_margin='0',
    block_label_padding='*spacing_sm *spacing_lg',
    block_label_radius='calc(*radius_lg - 1px) 0 calc(*radius_lg - 1px) 0',
    block_label_text_size='*text_sm',
    block_label_text_weight='400',
    block_title_background_fill='none',
    block_title_background_fill_dark='none',
    block_title_text_color='*neutral_500',
    block_title_text_color_dark='*neutral_200',
    block_title_padding='0',
    block_title_radius='none',
    block_title_text_weight='400',
    panel_border_width='0',
    panel_border_width_dark='0',
    checkbox_background_color_selected='*secondary_600',
    checkbox_background_color_selected_dark='*secondary_600',
    checkbox_border_color='*neutral_300',
    checkbox_border_color_dark='*neutral_700',
    checkbox_border_color_focus='*secondary_500',
    checkbox_border_color_focus_dark='*secondary_500',
    checkbox_border_color_selected='*secondary_600',
    checkbox_border_color_selected_dark='*secondary_600',
    checkbox_border_width='*input_border_width',
    checkbox_shadow='*input_shadow',
    checkbox_label_background_fill_selected='*checkbox_label_background_fill',
    checkbox_label_background_fill_selected_dark='*checkbox_label_background_fill',
    checkbox_label_shadow='none',
    checkbox_label_text_color_selected='*checkbox_label_text_color',
    input_background_fill='*neutral_100',
    input_border_color='*border_color_primary',
    input_shadow='none',
    input_shadow_dark='none',
    input_shadow_focus='*input_shadow',
    input_shadow_focus_dark='*input_shadow',
    slider_color='auto',
    slider_color_dark='auto',
    button_shadow='none',
    button_shadow_active='none',
    button_shadow_hover='none',
    button_primary_background_fill='*primary_200',
    button_primary_background_fill_hover='*button_primary_background_fill',
    button_primary_background_fill_hover_dark='*button_primary_background_fill',
    button_primary_text_color='*primary_600',
    button_secondary_background_fill='*neutral_200',
    button_secondary_background_fill_hover='*button_secondary_background_fill',
    button_secondary_background_fill_hover_dark='*button_secondary_background_fill',
    button_secondary_text_color='*neutral_700',
    button_cancel_background_fill_hover='*button_cancel_background_fill',
    button_cancel_background_fill_hover_dark='*button_cancel_background_fill'
)


#with gr.Blocks(theme=gr.themes.Default(primary_hue=gr.themes.colors.red, secondary_hue=gr.themes.colors.pink)) as demo_genretail:

def Load_images(urls = ["https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/2FRI-GALLERY-0-1.jpg?frz-width=643&context=bWFzdGVyfGltYWdlc3wyMDkwMjJ8aW1hZ2UvanBlZ3xpbWFnZXMvaGJjL2g4Ny85NTMwNjA5NTk4NDk0LmpwZ3xlM2Y0NmE0YWY3YjQ3MjcyMzRhNmEwOTE3MjY5OTMxOWU1OGQ4ZmU0MWM2ZTI3NDRkODAzNTllOWYyMzVkMTRm"]):
    images = []    
    for url in urls:
        images.append(Image.open(requests.get(url , stream=True).raw))
    return images

def get_select_index(imgs_set, evt: gr.SelectData):
    print(evt.index)
    if(evt.index in imgs_set):
        imgs_set.pop(imgs_set.index(evt.index))
    else:
        imgs_set.append(evt.index)
    print(imgs_set)
    return imgs_set


    
with gr.Blocks(theme=theme_demo) as demo_genretail:

    demo_genretail.theme = theme_demo
    
    header = gr.HTML(html)
#    gr.Markdown
    md = (    """

# **Gen** ret<strong><font color="red">AI</font></strong>l

Customer search:
* I search flower for **weading.**
* I search flower for **mother day.**
* I search flower for **valentine day.**


Company provide:
* I found the same product <strong><font color="red">contextualized</font></strong> to my search
    """)

    image_uri = "https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/2FRI-GALLERY-0-1.jpg?frz-width=643&context=bWFzdGVyfGltYWdlc3wyMDkwMjJ8aW1hZ2UvanBlZ3xpbWFnZXMvaGJjL2g4Ny85NTMwNjA5NTk4NDk0LmpwZ3xlM2Y0NmE0YWY3YjQ3MjcyMzRhNmEwOTE3MjY5OTMxOWU1OGQ4ZmU0MWM2ZTI3NDRkODAzNTllOWYyMzVkMTRm"
    images = []
  
    images.append(Image.open(requests.get(image_uri , stream=True).raw))

    #
    
    
    with gr.Row():
        with gr.Column():
            search = gr.Textbox(label="Search a product", placeholder="Try: Flower, tree, Bouquets, Roses... ")
            theme = gr.Textbox(label="Thema", placeholder="Try: weading, mother day, valentine days, Anniversary, Marriage, For pleasure, Congratulation, Birth...")
            
            language= gr.Textbox(label="Description language", placeholder="Try: FR, ES, NL, EN...")

            gallery_photos = gr.Gallery(
                label="Generated images", show_label=False, elem_id="gallery_id"
                ).style(columns=[2], rows=[2], object_fit="contain", height="auto")

            gallery_photos.select(
                get_select_index, images_selected_state, images_selected_state).then(
                None, images_selected_state, None, _js=js_pos_select)

            with gr.Accordion("Destination prompt advanced settings", open=False):
                gr.Markdown("Make changes to the destination prompt default settings")
                temperature = gr.Slider(0, 1, 0.3, label="temperature")
                top_k = gr.Slider(0, 40, 40, label="top_k")
                top_p = gr.Slider(0, 1, 1, label="top_p")
                max_output_tokens = gr.Slider(0, 1024, 700, label="max_output_tokens")
                
                images_selected_state = gr.JSON([], visible=False)





        with gr.Column():
            btn = gr.Button("Run")
            product_desc_prompt_out = gr.Textbox(label="Product description")
            product_composition_prompt_out = gr.Textbox(label="Product composition")
            
            product_images = gr.Gallery(label="Generated images", show_label=False, elem_id="gallery").style(columns=[3], rows=[2], object_fit="contain", height="auto")


    
    product_photo_prompt_out = "test"
            
    inputs_array = [search,
                    theme,
                    language,
                    temperature,                    
                    top_k,
                    top_p,
                    max_output_tokens]
    
    outputs_array = [product_desc_prompt_out ,
                     product_composition_prompt_out,
                     product_images
                     #, product_photo_prompt_out
#                                ,gr.Textbox(label="Image Generated Prompt")
#                         , product_images
                         ]
    
    
    demo_genretail.load(Load_images, None, gallery_photos)
    

    btn.click(fn=update, inputs=inputs_array, outputs=outputs_array )
    
    

    
    #, examples=["genAI/examples/B2BDP-GALLERY-0-1.jpg", "examples/B2BDP-GALLERY-0-1.jpg", "genAI/examples/B5CL-GALLERY-0-2.jpg"])
    
    
 
demo_genretail.launch(share=True, debug=True)

#if __name__ == "__main__":
    # Set debug=True in Colab for live debugging
    # To create a public link, set `share=True` in `launch()`.

#    demo_genretail.launch(share=True, debug=True)
#     demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7875
Running on public URL: https://ead153f79c12eeecca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


KeyError: 375

# 

In [19]:
import requests
import gradio as gr
from PIL import Image
import time

URL = "https://source.unsplash.com/random/500x500/?nature,fruit"

URL = "https://medias.interflora.fr/fstrz/r/s/c/medias.interflora.fr/medias/2FRI-GALLERY-0-1.jpg?frz-width=643&context=bWFzdGVyfGltYWdlc3wyMDkwMjJ8aW1hZ2UvanBlZ3xpbWFnZXMvaGJjL2g4Ny85NTMwNjA5NTk4NDk0LmpwZ3xlM2Y0NmE0YWY3YjQ3MjcyMzRhNmEwOTE3MjY5OTMxOWU1OGQ4ZmU0MWM2ZTI3NDRkODAzNTllOWYyMzVkMTRm"


js_pos_select = """
async (images_selected_state) => {
  const gallery = document.querySelector("#gallery_id")
  const buttons_thumbnails = gallery.querySelectorAll(".thumbnails > button");
  const buttons_large = gallery.querySelectorAll(".grid-container > button");
  buttons_thumbnails.forEach((btn, idx) => {
    if(images_selected_state.includes(idx)){
      btn.classList.add('selected-custom');
    }else{
      btn.classList.remove('selected-custom');
    }
  })
  buttons_large.forEach((btn, idx) => {
    if(images_selected_state.includes(idx)){
      btn.classList.add('selected-custom');
    }else{
      btn.classList.remove('selected-custom');
    }
  })
  return images_selected_state
}
"""
css = """
.selected-custom {
    --ring-color: red !important;
    transform: scale(0.9) !important;
    border-color: red !important;
}
"""
def get_random_images(n=1):
    images = []
    #  for i in range(n):
    images.append(Image.open(requests.get(URL , stream=True).raw))
    return images

def get_select_index(imgs_set, evt: gr.SelectData):
  print(evt.index)
  if(evt.index in imgs_set):
    imgs_set.pop(imgs_set.index(evt.index))
  else:
    imgs_set.append(evt.index)
  print(imgs_set)
  return imgs_set

with gr.Blocks(css=css) as demo:
    images_selected_state = gr.JSON([], visible=False)

    with gr.Column(variant="panel"):
        with gr.Row(variant="compact"):
            btn = gr.Button("Generate image").style(full_width=False)

        gallery_photos = gr.Gallery(
            label="Generated images", show_label=False, elem_id="gallery_id"
        ).style(columns=[4], rows=[2], object_fit="contain", height="auto")

    gallery_photos.select(
        get_select_index, images_selected_state, images_selected_state).then(
        None, images_selected_state, None, _js=js_pos_select)
    demo.load(get_random_images, None, gallery_photos)

demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://4d9d89fcb2e4ad26c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


0
[0]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7866 <> https://4d9d89fcb2e4ad26c5.gradio.live


In [2]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

In [3]:
import base64
import glob
import os
from datetime import datetime
from io import BytesIO

import requests
import torch
from google.cloud import aiplatform, storage
from PIL import Image


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str):
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(model_id, task):
    model_name = "stable-diffusion-v1"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_V100",
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))

In [4]:
from diffusers import StableDiffusionPipeline

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]

display(image)

2023-05-31 18:56:56.829234: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 18:57:00.478886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-31 18:57:00.479040: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2                                                                                             │
│    3 model_id = "runwayml/stable-diffusion-v1-5"                                                 │
│    4 pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)         │
│ ❱  5 pipe = pipe.to("cuda")                                                                      │
│    6                                                                                             │
│    7 prompt = "a photo of an astronaut riding a horse on mars"                                   │
│    8 image = pipe(prompt).images[0]                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/diffusers/pipelines/pipeline_utils.py:643 in to           │
│                                                                                                  │
│    640 │   │                                                                                     │
│    641 │   │   is_offloaded = pipeline_is_offloaded or pipeline_is_sequentially_offloaded        │
│    642 │   │   for module in modules:                                                            │
│ ❱  643 │   │   │   module.to(torch_device, torch_dtype)                                          │
│    644 │   │   │   if (                                                                          │
│    645 │   │   │   │   module.dtype == torch.float16                                             │
│    646 │   │   │   │   and str(torch_device) in ["cpu"]                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/modeling_utils.py:1811 in to                 │
│                                                                                                  │
│   1808 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1809 │   │   │   )                                                                             │
│   1810 │   │   else:                                                                             │
│ ❱ 1811 │   │   │   return super().to(*args, **kwargs)                                            │
│   1812 │                                                                                         │
│   1813 │   def half(self, *args):                                                                │
│   1814 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ /home/jupyter/.local/lib/python3.7/site-packages/torch/nn/modules/module.py:989 in to            │
│                                                                                                  │
│    986 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│    987 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│    988 │   │                                                                                     │
│ ❱  989 │   │   return self._apply(convert)                                                       │
│    990 │                                                                                         │
│    991 │   def register_backward_hook(                                                           │
│    992 │   │   self, hook: Callable[['Module', _grad_t, _gr